In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('train.csv')
df_train.set_index('row_id', inplace=True)

df_test = pd.read_csv('test.csv')

df_train

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag
row_id,,,,,,,,,,,,,,,,,
0,4,2019-07-16 00:00:00+08:00,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0
1,4,2019-07-16 00:00:00+08:00,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1
2,6,2019-07-16 00:00:00+08:00,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0
3,1,2019-07-16 00:00:00+08:00,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0
4,6,2019-07-16 00:00:00+08:00,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73534,6,2019-09-02 00:00:00+08:00,127613,39,24,36,279,0,1,1,0,0,0,0,0,0,0
73535,2,2019-09-02 00:00:00+08:00,127620,38,46,10,51,0,0,1,0,0,0,0,0,0,0
73536,2,2019-09-02 00:00:00+08:00,127696,32,Never open,Never login,Never checkout,0,0,0,0,0,0,0,0,0,0


In [3]:
df_users = pd.read_csv('users.csv')
df_users.set_index('user_id', inplace=True)
df_users.attr_2 = df_users.attr_2.astype(object)
df_users.attr_3 = df_users.attr_3.astype(object)
df_users.age = df_users.age.clip(0, 100)
df_users.age = df_users.age.apply(lambda x: 31 if np.isnan(x) else x)
df_users.domain = df_users.domain.apply(lambda x: 1 if x=='@gmail.com' else 0).astype(object)

df_users

,attr_1,attr_2,attr_3,age,domain
user_id,,,,,
0,NaN,1,0,31.0,1
1,1.0,1,2,50.0,1
2,NaN,1,0,31.0,0
3,NaN,1,0,31.0,1
4,1.0,1,2,33.0,1
...,...,...,...,...,...
127921,NaN,1,0,31.0,0
127922,1.0,1,0,20.0,1
127923,NaN,1,0,31.0,1


In [4]:
def prepDF(df, df_users):
    ceiling = {
        'last_open_day': 1000,
        'last_checkout_day':1500,
        'last_login_day':20000
          }
    df['last_open_day'] = df['last_open_day'].apply(
        lambda x: ceiling['last_open_day'] if x=='Never open' else int(x))
    df['last_checkout_day'] = df['last_checkout_day'].apply(
        lambda x: ceiling['last_checkout_day'] if x=='Never checkout' else int(x))
    df['last_login_day'] = df['last_login_day'].apply(
        lambda x: ceiling['last_login_day'] if x=='Never login' else int(x))
    df['country_code'] = df['country_code'].astype(object)
    
    df_merged = df.merge(df_users, left_on='user_id', right_index=True)

    features = [
        'country_code', 'subject_line_length',
        'last_open_day', 'last_login_day', 'last_checkout_day',
        'open_count_last_10_days', 'open_count_last_30_days',
        'open_count_last_60_days', 'login_count_last_10_days',
        'login_count_last_30_days', 'login_count_last_60_days',
        'checkout_count_last_10_days', 'checkout_count_last_30_days',
        'checkout_count_last_60_days', 'attr_2', 'attr_3', 'age'
               ]
    X = pd.get_dummies(df_merged[features], drop_first=True,
                       columns=['country_code', 'attr_2', 'attr_3'])
    if 'open_flag' in df_merged.columns:
        y = df_merged['open_flag']
    else:
        y = None
        
    return X, y

In [5]:
X, y = prepDF(df_train, df_users)

In [6]:
X

,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,...,country_code_3,country_code_4,country_code_5,country_code_6,country_code_7,attr_2_1.0,attr_3_1.0,attr_3_2.0,attr_3_3.0,attr_3_4.0
row_id,,,,,,,,,,,,,,,,,,,,,
0,44,19,6,18,0,2,4,12,43,99,...,0,1,0,0,0,1,0,1,0,0
1,44,9,4,8,2,9,17,18,48,90,...,0,1,0,0,0,1,0,1,0,0
2,49,14,5,5,0,4,12,24,69,119,...,0,0,0,1,0,1,0,0,0,0
3,49,49,9,53,0,0,1,9,23,69,...,0,0,0,0,0,1,0,1,0,0
4,49,227,6,221,0,0,0,2,5,5,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73534,39,24,36,279,0,1,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
73535,38,46,10,51,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
73536,32,1000,20000,1500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
# from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [8]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

Using TensorFlow backend.


In [9]:
# set seed for reproducibility
seed = 0
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=seed, stratify=y)

In [10]:
class_weights = {0:1, y.value_counts()[0]/y.value_counts()[1]}

In [11]:
cl1 = LogisticRegression(max_iter=1000, class_weight=class_weights,
                         solver='liblinear', random_state=seed)
cl2 = LinearSVC(dual=False, max_iter=10000, random_state=seed, class_weight=class_weights)
cl3 = DecisionTreeClassifier(max_features='log2', random_state=seed, class_weight=class_weights)
cl4 = RandomForestClassifier(max_features='log2', random_state=seed, class_weight=class_weights)
cl5 = GradientBoostingClassifier(max_features='log2', random_state=seed)

cl_list = [cl1, cl2, cl3, cl4, cl5]

In [12]:
penalty = ['l1', 'l2']
C = np.logspace(-5, 5, num=15)
max_depth = [3, 4, 5, 6, 7]
n_estimators = [20, 40, 64, 128, 256]
learning_rate = [0.005, 0.01, 0.05, 0.1]
subsample_list = [0.3, 0.5, 0.7, 1]
k = [5, 10, 15, 20]


params = {'decisiontreeclassifier__max_depth': n_estimators,
          'randomforestclassifier__n_estimators': n_estimators,
          'randomforestclassifier__max_depth': max_depth,
          'gradientboostingclassifier__max_depth': max_depth,
          'gradientboostingclassifier__n_estimators': n_estimators,
          'gradientboostingclassifier__learning_rate': learning_rate,
          'gradientbosstingclassifier__subsample_list': subsample_list,
          'logisticregression__C': C,
          'logisticregression__penalty': penalty,
          'linearsvc__penalty': penalty,
          'linearsvc__C': C,}

param_grid = []
for cl in cl_list:
    cl_params = {}#'sampler__k_neighbors':k}
    for p, val in params.items():
        name = p.split('__')[0]
        parameter = p.split('__')[1]

        if name == cl.__class__.__name__.lower():
            cl_params[f'model__{parameter}'] = val

    param_grid.append(cl_params)

In [13]:
def ml_tuner(X, y, estimators, params, scaler, sampler, scorer):
    models = {}
    for cl, p in zip(estimators, params):
        print(cl.__class__.__name__)
        est = Pipeline(
            [('sampler', sampler), ('scaler', scaler), ('model', cl)]
        )
        gs_cv = GridSearchCV(est, param_grid=p,
                             n_jobs=-1, scoring=scorer,
                             cv=10, verbose=1)
        gs_cv.fit(X, y)        
        models[cl.__class__.__name__] = gs_cv
    return models

In [14]:
scorer = metrics.make_scorer(metrics.matthews_corrcoef)

In [15]:
tuned_models = ml_tuner(
    X_train, y_train, cl_list, param_grid,
    MinMaxScaler(), SMOTE(random_state=seed), scorer
)

LogisticRegression
Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.0min finished


LinearSVC
Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 12.5min finished


DecisionTreeClassifier
Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   20.1s finished


RandomForestClassifier
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  8.9min finished


GradientBoostingClassifier
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 22.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 41.6min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 53.4min finished


In [18]:
tuned_models.keys()

dict_keys(['LogisticRegression', 'LinearSVC', 'DecisionTreeClassifier', 'RandomForestClassifier', 'GradientBoostingClassifier'])

In [19]:
gs_scores = {}

gs_scores['names'] = [name for name in tuned_models.keys()]
gs_scores['CV_Score'] = [m.best_score_ for m in tuned_models.values()]
gs_scores['TestScore'] = [m.score(X_test, y_test) for m in tuned_models.values()]
gs_scores['MCC'] = [scorer(m, X_test, y_test) for m in tuned_models.values()]

df_scores = pd.DataFrame(gs_scores)
df_scores

,names,CV_Score,TestScore,MCC
0,LogisticRegression,0.229749,0.231244,0.231244
1,LinearSVC,0.190959,0.197555,0.197555
2,DecisionTreeClassifier,0.311408,0.316755,0.316755
3,RandomForestClassifier,0.235659,0.239493,0.239493
4,GradientBoostingClassifier,0.482197,0.481314,0.481314


In [22]:
X_holdout, _ = prepDF(df_test, df_users)

In [25]:
y_pred = tuned_models['GradientBoostingClassifier'].predict(X_holdout)

In [31]:
df_submission = pd.DataFrame({
    'row_id':df_test['row_id'].values,
    'open_flag':y_pred
})
df_submission.to_csv('submission.csv', index=False)